## Sort the model first

In [1]:
import os

os.environ['YOLO_VERBOSE'] = 'False'

In [2]:
import torch
from ultralytics import YOLO
from torch import nn

In [3]:
from utils import model_sorting, utils
from channel_pruning import pruner

In [4]:
model = YOLO('tsr.pt').cuda()

In [5]:
sorted_model = model_sorting.sort_model(model)

In [6]:
sorted_model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## Pruning

In [7]:
pruned_model=pruner.prune_layer(sorted_model,p_ratio=0.3)        

Conv Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
SPPF Weight transfer complete.
None
C2f Weight transfer complete.
135
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Sequential weight transfer complete.


In [8]:
pruned_model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 11, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (1): Conv(
        (conv): Conv2d(11, 22, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(22, 22, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cv2): Conv(
          (conv): Conv2d(33, 22, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
     

In [9]:
@torch.inference_mode()
def evaluate(
  model: nn.Module,
) -> float:
  
  metrics = model.val(data = 'road_sign_data.yaml', split = 'val')
  acc = metrics.results_dict['metrics/mAP50(B)']

  return acc

In [10]:
model_accuracy = evaluate(pruned_model)

/home/dell/Yolov8_Optimization/venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [13]:
def fine_tune(model: nn.Module):
  model.train(data='road_sign_data.yaml', epochs=10, imgsz=640,device = '0')

In [14]:
fine_tune(pruned_model)

100%|██████████| 6.25M/6.25M [00:01<00:00, 3.97MB/s]
/home/dell/Yolov8_Optimization/venv/lib/python3.8/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/dell/Yolov8_Optimization/venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [17]:
utils.get_model_size(pruned_model)

42365472

In [18]:
utils.get_model_size(model)

96449056